In [1]:
import os
import requests
from dotenv import load_dotenv
import base64
import json
import numpy as np
import pandas as pd
from snowflake.connector.pandas_tools import write_pandas
from utils.sf_conn import Config
from utils.assistant_api import Spotify_Assistant
from utils.prompt import PROMPT
import time
load_dotenv()

True

In [2]:
sf_config = Config()
create_conn = sf_config.create_sf_conn()

In [3]:
cursor = create_conn.cursor()
query = "SELECT * FROM MAIN_SPOTIFY"
spotify_df = cursor.execute(query).fetch_pandas_all()
spotify_df.head(3)

,TRACK_ID,PLAYLIST_ID,PLAYLIST_NAME,TRACK_NAME,TRACK_POPULARITY,ARTISTS,GENRES,DANCEABILITY,DURATION_MS,ENERGY,INSTRUMENTAL,SONG_KEY,LIVENESS,LOUDNESS,SPEECHINESS
0,1qE47wUKG2juJwPoLqg4C9,37i9dQZF1EQp9BVPsNVof1?si=b0e0182f48ea461c37i9...,Dance/Electronic Mix,Lean On (feat. MØ & DJ Snake),65,"Major Lazer, MØ, DJ Snake","dance pop, edm, electro house, moombahton, pop...",0.723,176561,0.809,0.00123,7,0.5650,-3.081,0.0625
1,0K6iKgHPPhAb4Acmg9aD77,37i9dQZF1EQp9BVPsNVof1?si=b0e0182f48ea461c37i9...,Dance/Electronic Mix,On My Love,75,"Zara Larsson, David Guetta","pop, scandipop, swedish electropop, swedish po...",0.492,222912,0.688,0.00290,10,0.0881,-4.989,0.0460
2,1gihuPhrLraKYrJMAEONyc,37i9dQZF1EQp9BVPsNVof1?si=b0e0182f48ea461c37i9...,Dance/Electronic Mix,Feel So Close - Radio Edit,84,Calvin Harris,"dance pop, edm, electro house, house, pop, pop...",0.707,206413,0.924,0.00703,7,0.2040,-2.842,0.0310


In [ ]:
spotify_csv = spotify_df.to_csv('assistant_data/spotify_main_file')

In [4]:
def wait_until_run_status_completed(assistant, run_id):
    start_time = time.time() # starting a timer in case, the while loop gets stuck
    while True:
        (status := assistant.run_status(run_id))
        if status == "completed":
            break
        elif status in {"in_progress", "queued"}:
            cur_time = time.time()
            time_elasped = cur_time - start_time
            if time_elasped >= 600: # break the loop after it's been running for 10 mins
                break
            continue
        else:
            break
    assert status == "completed"

In [ ]:
# def upload_csv_file(file_path):
#     api_key =  os.getenv('OPENAI_API_KEY')
#     openai.api_key = api_key
#     with open(file_path, "rb") as file:
#         uploaded_file = openai.files.create(
#             file=file,
#             purpose='assistants'
#         )
#     return uploaded_file.id

In [5]:
assistant = Spotify_Assistant(assistant_id='asst_afQAdlE40WllyVRI7mDZBjha')
# assistant.upload_csv_file('assistant_data/spotify_main_file')

# print(assistant.create_assistant(file_ids=[file_id]))
assistant.attach_message_to_thread('What is the least popular song in the file?')
run = assistant.start_run(override_prompt=PROMPT)
assistant_status = wait_until_run_status_completed(assistant=assistant, run_id = run.id)
response = assistant.assistant_answer()
print(response)





The least popular song in the file is "Everclear" by Crumb Pit, with a TRACK_POPULARITY score of 0. This indicates it's the least popular among the songs listed in the file based on the metric used.


In [6]:
! pip list

Package                    Version
-------------------------- ------------
altair                     5.2.0
annotated-types            0.6.0
anyio                      4.2.0
appnope                    0.1.2
asn1crypto                 1.5.1
asttokens                  2.0.5
attrs                      23.2.0
blinker                    1.7.0
cachetools                 5.3.2
certifi                    2023.11.17
cffi                       1.16.0
charset-normalizer         3.3.2
click                      8.1.7
comm                       0.1.2
cryptography               41.0.7
debugpy                    1.6.7
decorator                  5.1.1
distro                     1.9.0
exceptiongroup             1.0.4
executing                  0.8.3
filelock                   3.13.1
gitdb                      4.0.11
GitPython                  3.1.41
h11                        0.14.0
httpcore                   1.0.2
httpx                      0.26.0
idna                       3.6
importlib-metadata     